In [ ]:
#Import packages
import osmnx as ox
import folium
import json
import pandas as pd

In [7]:
#Define parameters
place = "University of Tennessee, Knoxville, TN"

In [ ]:
#Geocode
aoi = ox.geocoder.geocode_to_gdf(place)

In [9]:
#Show interactive map
aoi.explore()

In [11]:
#Get buildings
tags = {"building": True}
buildings = ox.features.features_from_place(place, tags).reset_index()
buildings

,element,id,geometry,addr:state,building,ele,gnis:feature_id,name,source,wikidata,...,denomination,religion,residential,network,public_transport,railway,train,smoking,type,height
0,node,368157840,POINT (-83.93074 35.95425),TN,yes,291,1953958,Melrose Hall,USGS Geonames,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,8907136455,POINT (-83.94031 35.95072),NaN,university,NaN,NaN,University Commons,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,8907329041,POINT (-83.92493 35.95688),NaN,university,NaN,NaN,Nuclear Engineering Building,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,8907329042,POINT (-83.92445 35.95699),NaN,university,NaN,NaN,Science & Engineering,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,relation,1637098,"POLYGON ((-83.92458 35.95599, -83.92449 35.955...",TN,yes,NaN,NaN,Neyland Stadium,NaN,Q1984022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multipolygon,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,way,1314690970,"POLYGON ((-83.93227 35.95696, -83.93216 35.957...",NaN,yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,way,1320664892,"POLYGON ((-83.92862 35.94583, -83.92858 35.945...",NaN,yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,way,1329395659,"POLYGON ((-83.94111 35.94786, -83.94064 35.948...",TN,yes,NaN,NaN,Agricultural Sciences and Natural Resources,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,way,1329395660,"POLYGON ((-83.939 35.94406, -83.93889 35.944, ...",NaN,pavilion,NaN,NaN,UT Gardens Pavilion,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN


In [26]:
#Filter out buildings without a name
buildings_filtered = buildings.dropna(subset=['name'])

# Check what values exist in the 'element' column
buildings_filtered['element'].unique()

# Then filter out 'node'
buildings_filtered = buildings_filtered[buildings_filtered['element'] != 'node']

In [27]:
# Create a map centered around the place
m = folium.Map(location=[aoi.geometry.centroid.y.iloc[0], aoi.geometry.centroid.x.iloc[0]], zoom_start=16)

# Add building polygons to the map with name as a label
for _, row in buildings_filtered.iterrows():
    folium.GeoJson(
        row['geometry'],
        tooltip=row['name']
    ).add_to(m)

# Display the map
m

/var/folders/vy/727_8hxd44n4jrxb56wtgxl00000gn/T/ipykernel_10447/1339505623.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[aoi.geometry.centroid.y.iloc[0], aoi.geometry.centroid.x.iloc[0]], zoom_start=16)


To use these shapes in an ArcGIS visual in Power BI, you need to create a columm with the coordinates in an EsriJSON format, which looks like this: `{"rings": [[[x1,y1], [x2,y2], ...]]}`

In [ ]:
# Convert geometry to EsriJSON rings format
def geometry_to_esrijson_rings(geometry):
    """Convert a shapely geometry to EsriJSON rings format"""
    if geometry.geom_type == 'Polygon':
        coords = list(geometry.exterior.coords)
        rings = [[[coord[0], coord[1]] for coord in coords]]
        return {"rings": rings}
    elif geometry.geom_type == 'MultiPolygon':
        # Handle multiple polygons
        rings = []
        for poly in geometry.geoms:
            coords = list(poly.exterior.coords)
            rings.append([[coord[0], coord[1]] for coord in coords])
        return {"rings": rings}
    return None

# Create the table with name and coordinates columns
table_data = []
for idx, row in buildings_filtered.iterrows():
    name = row.get('name', '')  # Get name, empty string if NaN
    coordinates = geometry_to_esrijson_rings(row.geometry)
    
    if coordinates:  # Only add if we have valid coordinates
        table_data.append({
            'name': name,
            'coordinates': json.dumps(coordinates)  # Convert to JSON string
        })

# Create DataFrame
coordinates_table = pd.DataFrame(table_data)

# Display the table
coordinates_table

,name,coordinates
0,Neyland Stadium,"{""rings"": [[[-83.9245753, 35.9559889], [-83.92..."
1,Humanities and Social Sciences,"{""rings"": [[[-83.9295056, 35.953948], [-83.929..."
2,University Printing & Mail,"{""rings"": [[[-83.9286417, 35.9463879], [-83.92..."
3,UT Warehouse,"{""rings"": [[[-83.9291948, 35.9462953], [-83.92..."
4,UT Warehouse,"{""rings"": [[[-83.9302013, 35.94693], [-83.9299..."
...,...,...
122,Tyson House,"{""rings"": [[[-83.9318048, 35.9558728], [-83.93..."
123,Three Rivers Rambler,"{""rings"": [[[-83.9404796, 35.9529348], [-83.94..."
124,Neyland Parking Garage G10,"{""rings"": [[[-83.9241743, 35.952325], [-83.924..."
125,Agricultural Sciences and Natural Resources,"{""rings"": [[[-83.9411055, 35.9478623], [-83.94..."


In [31]:
# Save as CSV
coordinates_table.to_csv("buildings_esrijson.csv", index=False)